In [1]:
from pyspark.sql import SQLContext
from pyspark.context import SparkContext

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
df=sqlContext.read.load('daily_weather.csv', format='com.databricks.spark.csv', header='true', inferSchema='true')

In [4]:
df

DataFrame[number: int, air_pressure_9am: double, air_temp_9am: double, avg_wind_direction_9am: double, avg_wind_speed_9am: double, max_wind_direction_9am: double, max_wind_speed_9am: double, rain_accumulation_9am: double, rain_duration_9am: double, relative_humidity_9am: double, relative_humidity_3pm: double]

In [5]:
import pandas as pd
import numpy as np

In [6]:
df1 = pd.read_csv('daily_weather.csv')
df1.head()

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,1,917.347688,71.403843,101.935179,2.443009,140.471549,3.533324,0.0,0.0,24.328697,19.426597
2,2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000
3,3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547
4,4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000


In [14]:
df.toPandas().describe().transpose()

,count,mean,std,min,25%,50%,75%,max
number,1095.0,547.000000,316.243577,0.000000,273.500000,547.000000,820.500000,1094.000000
air_pressure_9am,1092.0,918.882551,3.184161,907.990000,916.550000,918.921045,921.160073,929.320000
air_temp_9am,1090.0,64.933001,11.175514,36.752000,57.281000,65.715479,73.450974,98.906000
avg_wind_direction_9am,1091.0,142.235511,69.137859,15.500000,65.972506,166.000000,191.000000,343.400000
avg_wind_speed_9am,1092.0,5.508284,4.552813,0.693451,2.248768,3.871333,7.337163,23.554978
max_wind_direction_9am,1092.0,148.953518,67.238013,28.900000,76.553003,177.300000,201.233153,312.200000
max_wind_speed_9am,1091.0,7.019514,5.598209,1.185578,3.067477,4.943637,8.947760,29.840780
rain_accumulation_9am,1089.0,0.203079,1.593952,0.000000,0.000000,0.000000,0.000000,24.020000
rain_duration_9am,1092.0,294.108052,1598.078779,0.000000,0.000000,0.000000,0.000000,17704.000000
relative_humidity_9am,1095.0,34.241402,25.472067,6.090000,15.092243,23.179259,45.400000,92.620000


In [15]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141293575|
| stddev|11.175514003266809|
|    min|            36.752|
|    max|            98.906|
+-------+------------------+



In [16]:
df.count()

1095

In [26]:
removeAllDF = df.na.drop()
removeAllDF['air_temp_9am']
# removeAllDF.describe(['air_temp_9am']).show()

Column<b'air_temp_9am'>

In [19]:
removeAllDF.count()

1064

In [20]:
from pyspark.sql.functions import avg
imputeDF = df

In [23]:
for x in imputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x,meanValue)
    imputeDF = imputeDF.na.fill(meanValue,[x])

number 545.0018796992481
air_pressure_9am 918.9031798643798
air_temp_9am 65.02260949566728
avg_wind_direction_9am 142.3067556490319
avg_wind_speed_9am 5.485793050719923
max_wind_direction_9am 148.48042413302613
max_wind_speed_9am 6.99971365887782
rain_accumulation_9am 0.1820234765093985
rain_duration_9am 266.3936973989333
relative_humidity_9am 34.07743985330167
relative_humidity_3pm 35.14838093288156


In [24]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141293575|
| stddev|11.175514003266809|
|    min|            36.752|
|    max|            98.906|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058226329|
| stddev|11.149948200010943|
|    min|            36.752|
|    max|            98.906|
+-------+------------------+

